In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from collections import defaultdict

Проверка описания

In [19]:
desc = pd.read_csv("E:/!sofi/Datasets/datasets/hse-mlds-project-year2/data/Женщинам_Футболки&и&топы/descriptions.csv")

In [20]:
desc

,item,description
0,0.png,Представляем вам женскую базовую бельевую спор...
1,1.png,Хлопковый стильный короткий кроп-топ в рубчик ...
2,2.png,Кроп топ женский на тонких бретелях - незамени...
3,3.png,Футболка женская плотная свободного кроя оверс...
4,4.png,Футболка Хизер от бренда Jelika – универсальна...
5,5.png,Чёрная футболка superoversize с принтом выпол...
6,6.png,"Топ двойной длинный - это трендовая вещь, кото..."
7,7.png,Футболка женская оверсайз бренда ТЕЛОДВИЖЕНИЯ ...
8,8.png,Добро пожаловать в нашу студию печати WAKA WAK...
9,9.png,Футболка с округлым вырезом давно стала неотъе...


____

Сбор статистики по спаршенным фотографиям

In [212]:
root_dir = Path("C:/temp/!hse-mlds-project-year2/data")

In [213]:
# Кол-во категорий
folders = os.listdir(root_dir)
len(folders)

1580

In [4]:
df_dict = {col: [] for col in ["card", "feedbacks"]}

for cat in folders:
    path = root_dir / cat
    path_card = path / "card"
    path_feedbacks = path / "feedbacks"

    # кол-во загруженных фоток
    load_card = len(sorted(path_card.glob('*')))
    load_feedbacks = len(sorted(path_feedbacks.glob('*')))
    
    if load_feedbacks < 5:
        print(f"{load_feedbacks} Feedbacks in {cat}")

    df_dict["card"].append(load_card)
    df_dict["feedbacks"].append(load_feedbacks)

3 Feedbacks in Автотовары_Запчасти&на&легковые&автомобили_Ходовая&часть_Рычаги&подвески
3 Feedbacks in Автотовары_Мойки&высокого&давления&и&аксессуары_Подметальные&машины
2 Feedbacks in Автотовары_Мойки&высокого&давления&и&аксессуары_Снегоуборщики
3 Feedbacks in Дом_Кухня_Магниты_Открывалки
4 Feedbacks in Дом_Кухня_Магниты_С&термометром
2 Feedbacks in Дом_Кухня_Магниты_Спорт
0 Feedbacks in Дом_Освещение_Лифты&для&люстр
4 Feedbacks in Дом_Предметы&интерьера_Картины&и&постеры_Картины_Аниме
3 Feedbacks in Дом_Предметы&интерьера_Картины&и&постеры_Картины_Карты
1 Feedbacks in Книги_Букинистика_Детективы
3 Feedbacks in Книги_Букинистика_Другие&жанры
0 Feedbacks in Книги_Букинистика_Историческая&и&военная&литература
1 Feedbacks in Книги_Букинистика_Любовные&романы
2 Feedbacks in Книги_Букинистика_Познавательная&литература&для&детей
2 Feedbacks in Книги_Букинистика_Поэзия
0 Feedbacks in Книги_Букинистика_Приключенческая&литература
2 Feedbacks in Книги_Букинистика_Проза
0 Feedbacks in Книги_Бук

In [5]:
df = pd.DataFrame(df_dict)
df.head()

,card,feedbacks
0,20,16
1,20,10
2,20,17
3,20,20
4,20,18


In [6]:
df.describe()

,card,feedbacks
count,1668.000000,1668.000000
mean,19.976619,17.665468
std,0.610883,4.248104
min,1.000000,0.000000
25%,20.000000,18.000000
50%,20.000000,19.000000
75%,20.000000,20.000000
max,20.000000,20.000000


In [7]:
df.feedbacks.value_counts()

feedbacks
20    750
19    360
18    177
17     84
16     46
15     38
13     27
14     27
0      22
11     17
10     17
12     16
3      13
1      12
8      12
9      12
2      11
5       8
7       7
6       7
4       5
Name: count, dtype: int64

In [8]:
df.card.value_counts()

card
20    1663
19       2
18       1
4        1
1        1
Name: count, dtype: int64

___

Проверка на дубликаты ссылок

In [180]:
categories = pd.read_csv("wb_categories_full_clear.csv")
categories.head()

,category_1,category_2,category_3,category_4,category_5,url
0,Автотовары,Мойки высокого давления и аксессуары,Цепные пилы,NaN,NaN,https://www.wildberries.ru/catalog/dom/dachniy...
1,Автотовары,Автокосметика и автохимия,Присадки,NaN,NaN,https://www.wildberries.ru/catalog/avtotovary/...
2,Автотовары,Автокосметика и автохимия,Смазки автомобильные,NaN,NaN,https://www.wildberries.ru/catalog/avtotovary/...
3,Автотовары,Автокосметика и автохимия,Уход за стеклами и фарами,NaN,NaN,https://www.wildberries.ru/catalog/aksessuary/...
4,Автотовары,Автокосметика и автохимия,Уход за салоном,NaN,NaN,https://www.wildberries.ru/catalog/aksessuary/...


In [181]:
df_dupl = categories[categories.duplicated(['url'], keep=False)]
df_dupl

,category_1,category_2,category_3,category_4,category_5,url
0,Автотовары,Мойки высокого давления и аксессуары,Цепные пилы,NaN,NaN,https://www.wildberries.ru/catalog/dom/dachniy...
13,Автотовары,Автоэлектроника и навигация,NaN,NaN,NaN,https://www.wildberries.ru/catalog/elektronika...
14,Автотовары,Аккумуляторы и сопутствующие товары,NaN,NaN,NaN,https://www.wildberries.ru/catalog/aksessuary/...
48,Автотовары,Мойки высокого давления и аксессуары,Аксессуары и запчасти для садовой техники,NaN,NaN,https://www.wildberries.ru/catalog/dom/dachniy...
49,Автотовары,Мойки высокого давления и аксессуары,Садовые измельчители,NaN,NaN,https://www.wildberries.ru/catalog/dom/dachniy...
...,...,...,...,...,...,...
1647,Электроника,Смартфоны и телефоны,Стационарные телефоны,NaN,NaN,https://www.wildberries.ru/catalog/elektronika...
1648,Электроника,Смартфоны и телефоны,Смартфоны,NaN,NaN,https://www.wildberries.ru/catalog/elektronika...
1649,Электроника,Смартфоны и телефоны,Планшеты,NaN,NaN,https://www.wildberries.ru/catalog/elektronika...
1650,Электроника,Смартфоны и телефоны,Мобильные телефоны,NaN,NaN,https://www.wildberries.ru/catalog/elektronika...


In [182]:
df_dupl.index

Index([   0,   13,   14,   48,   49,   50,   51,   52,   68,   69,
       ...
       1640, 1641, 1642, 1643, 1646, 1647, 1648, 1649, 1650, 1651],
      dtype='int64', length=156)

In [183]:
duplicates = defaultdict(list)

for ind in df_dupl.index:
    url = df_dupl.loc[ind]["url"]
    duplicates[url].append(ind)

In [184]:
print(len(duplicates))
duplicates

69


defaultdict(list,
            {'https://www.wildberries.ru/catalog/dom/dachniy-sezon/sadovaya-tehnika/tsepnye-pily': [0,
              1440,
              1480],
             'https://www.wildberries.ru/catalog/elektronika/avtoelektronika': [13,
              259],
             'https://www.wildberries.ru/catalog/aksessuary/avtotovary/akkumulyatory-i-soputstvuyushchie-tovary': [14,
              1588],
             'https://www.wildberries.ru/catalog/dom/dachniy-sezon/sadovaya-tehnika/aksessuary-i-zapchasti-dlya-sadovoy-tehniki': [48,
              1473,
              1475],
             'https://www.wildberries.ru/catalog/dom/dachniy-sezon/sadovaya-tehnika/sadovye-izmelchiteli': [49,
              1470,
              1476],
             'https://www.wildberries.ru/catalog/dom/dachniy-sezon/sadovaya-tehnika/motobury-i-aksessuary-k-nim': [50,
              1478,
              1498],
             'https://www.wildberries.ru/catalog/dom/dachniy-sezon/sadovaya-tehnika/kultivatory-i-motoblo

In [185]:
categories["url"][207], categories["url"][976]

('https://www.wildberries.ru/catalog/podarki/detyam/shariki-i-upakovka',
 'https://www.wildberries.ru/catalog/knigi/bukinistika/lyubovnye-romany')

In [186]:
df_duplicates = pd.DataFrame(duplicates.items(), columns=["url", "index"])

In [187]:
#df_duplicates.to_csv("wb_categories_dublicates.csv", index=False)

In [24]:
df = pd.DataFrame({
    'brand': ['Yum Yum', 'Yum Yum', 'Indomie', 'Indomie', 'Indomie'],
    'style': ['cup', 'cup', 'cup', 'pack', 'pack'],
    'rating': [4, 4, 3.5, 15, 5]
})
df

,brand,style,rating
0,Yum Yum,cup,4.0
1,Yum Yum,cup,4.0
2,Indomie,cup,3.5
3,Indomie,pack,15.0
4,Indomie,pack,5.0


In [25]:
df[df.duplicated(['style'], keep=False)]

,brand,style,rating
0,Yum Yum,cup,4.0
1,Yum Yum,cup,4.0
2,Indomie,cup,3.5
3,Indomie,pack,15.0
4,Indomie,pack,5.0


___

удаление дубликатов

In [207]:
import shutil
from itertools import chain

In [188]:
df_duplicates_choose = pd.read_csv("E://!docum//!HSE магистратура//Задание диплом//wb_categories_dublicates.csv")
#df_duplicates = pd.read_csv("wb_categories_dublicates.csv")

df_duplicates_choose = pd.concat([df_duplicates_choose[:12],
                                  pd.DataFrame([np.nan], columns=["choose_category"]),
                                  df_duplicates_choose[12:]], axis=0)
df_duplicates_choose.reset_index(drop=True, inplace=True)

df_duplicates = pd.concat([df_duplicates, df_duplicates_choose], axis=1)
#df_duplicates["delete"] = df_duplicates["index"].copy(deep=True)

In [189]:
df_duplicates.apply(lambda x : x["index"].remove(x["choose_category"]) if (str(x["choose_category"]) != "nan") else x["index"], axis=1);

In [190]:
df_duplicates

,url,index,choose_category
0,https://www.wildberries.ru/catalog/dom/dachniy...,"[0, 1440]",1480.0
1,https://www.wildberries.ru/catalog/elektronika...,[259],13.0
2,https://www.wildberries.ru/catalog/aksessuary/...,[1588],14.0
3,https://www.wildberries.ru/catalog/dom/dachniy...,"[48, 1473]",1475.0
4,https://www.wildberries.ru/catalog/dom/dachniy...,"[49, 1470]",1476.0
...,...,...,...
64,https://www.wildberries.ru/catalog/elektronika...,[1537],1642.0
65,https://www.wildberries.ru/catalog/elektronika...,[1549],1634.0
66,https://www.wildberries.ru/catalog/elektronika...,[1558],1633.0
67,https://www.wildberries.ru/catalog/elektronika...,[1559],1632.0


In [200]:
root_dir = Path("C:/temp/!hse-mlds-project-year2/data")

In [201]:
folders_name = []

for i in range(len(categories)):
    if str(categories.iloc[i]["category_5"]) == "nan":
        if str(categories.iloc[i]["category_4"]) == "nan":
            if str(categories.iloc[i]["category_3"]) == "nan":
                if str(categories.iloc[i]["category_2"]) == "nan":
                    cat = list(categories.iloc[i][:1])
                else:
                    cat = list(categories.iloc[i][:2])
            else:
                cat = list(categories.iloc[i][:3])
        else:
            cat = list(categories.iloc[i][:4])
    else:
        cat = list(categories.iloc[i][:5])
    folder = "_".join(["&".join(i.split()) for i in cat])
    folders_name.append(folder)

In [209]:
delete_index = list(chain(*list(df_duplicates["index"])))

In [214]:
len(delete_index)

88

In [211]:
for ind in delete_index:
    shutil.rmtree(root_dir / folders_name[ind])